In [19]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np
import gc
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(7)

In [20]:
def bootstrap_auc(y_true, y_pred, n=1000, sample_size=None):
    """
    Generate `n` bootstrap samples, evaluating `func`
    at each resampling. `bootstrap` returns a function,
    which can be called to obtain confidence intervals
    of interest.
    """
    simulations = list()
    if sample_size is None:
        sample_size = len(y_pred)
    num_tried = 0
    while len(simulations) < n and num_tried < 10000:
        num_tried += 1
        iteridx = np.random.choice(range(len(y_pred)), size=sample_size, replace=True)
        iterypred = [np.array(y_pred)[idx] for idx in iteridx]
        iterytrue = [np.array(y_true)[idx] for idx in iteridx]
        if len(set(iterytrue)) < 2:
            continue
        simulations.append(roc_auc_score(iterytrue, iterypred))
    simulations.sort()
    def ci(p):
        """
        Return 2-sided symmetric confidence interval specified
        by p.
        """
        u_pval = (1+p)/2.
        l_pval = (1-u_pval)
        l_indx = int(np.floor(n*l_pval))
        u_indx = int(np.floor(n*u_pval))
        return(np.mean(simulations), np.abs(simulations[l_indx]-simulations[u_indx])/2)
    result = ci(.95)
    return result
    

# Calculating AUC per split

In [18]:
splits = []
for i in [1,2,3,4,5]:
    print("Split {}".format(i))
    cxr_df = pd.read_csv('model_files/cxr_run_{}/cxr_test_result.csv'.format(i))
    
    cxr_csv = pd.read_csv('test_dfs/cxr_test_df.csv')
    
    nih_df = pd.read_csv('model_files/nih_run_{}/nih_test_result.csv'.format(i))
    
    nih_csv = pd.read_csv('test_dfs/nih_test_df.csv')
    
    cxp_df = pd.read_csv('model_files/cxp_run_{}/cxp_test_result.csv'.format(i))

    cxp_csv = pd.read_csv('test_dfs/cxp_test_df.csv')
    cxr_df = cxr_df[['Path', 'Pneumothorax']]
    cxr_df.columns = ['Path', 'Pneumothorax_pred']
    cxr_df = cxr_df.merge(cxr_csv, on=['Path'])
    nih_df = nih_df[['Path', 'Pneumothorax']]
    nih_df.columns = ['Path', 'Pneumothorax_pred']
    nih_df = nih_df.merge(nih_csv, on=['Path'])
    nih_df['study_id'] = nih_df.apply(lambda row: str(row['Patient ID'])+'/'+str(row['Follow-up #']), axis=1)

    cxp_df = cxp_df[['Path', 'Pneumothorax']]
    cxp_df.columns = ['Path', 'Pneumothorax_pred']
    cxp_df = cxp_df.merge(cxp_csv, on=['Path'])
    cxp_df['study_id'] = cxp_df.apply(lambda row: row['Path'].split('chexpert_full/train/')[1].split('/view')[0], axis=1)

    split = []
    cxr = cxr_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').max()[['Pneumothorax', 'Pneumothorax_pred']]
    cxp = cxp_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').max()[['Pneumothorax', 'Pneumothorax_pred']]
    nih = nih_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').max()[['Pneumothorax', 'Pneumothorax_pred']]
    
    gt = cxr['Pneumothorax'].values
    pred = cxr['Pneumothorax_pred'].values
    aucs = []
    idx1 = np.arange(len(cxr['Pneumothorax'].values))
    idx2 = np.arange(len(cxp['Pneumothorax'].values))
    idx3 = np.arange(len(nih['Pneumothorax'].values))

    for ix in range(1000):
        np.random.shuffle(idx1)
        np.random.shuffle(idx2)
        np.random.shuffle(idx3)

        auc1 = roc_auc_score(cxr['Pneumothorax'].values[idx1[0:645]], cxr['Pneumothorax_pred'].values[idx1[0:645]])
        auc2 = roc_auc_score(cxp['Pneumothorax'].values[idx2[0:645]], cxp['Pneumothorax_pred'].values[idx2[0:645]])
        auc3 = roc_auc_score(nih['Pneumothorax'].values[idx3[0:645]], nih['Pneumothorax_pred'].values[idx3[0:645]])
        auc = [auc1, auc2, auc3]
        aucs.append(np.max(auc)-np.min(auc))
        
    print("mean auc difference:", np.mean(aucs))

Split 1
mean auc difference: 0.06426156200292223
Split 2
mean auc difference: 0.05469859493277769
Split 3
mean auc difference: 0.058323664186062704
Split 4
mean auc difference: 0.05754911143571424
Split 5
mean auc difference: 0.05886680539210924


# Preprocessing study level scores

In [23]:
cxr_dfs = []
for i in [1,2,3,4,5]:
    cxr_dfs.append(pd.read_csv('model_files/cxr_run_{}/cxr_test_result.csv'.format(i)))
cxr_df = pd.concat(cxr_dfs)[['Path', 'Pneumothorax']]
cxr_df.columns = ['Path', 'Pneumothorax_pred']
cxr_df = cxr_df.groupby(cxr_df['Path']).mean()
cxr_test_csv = pd.read_csv('cxr_test_df.csv')
cxr_df = cxr_df.merge(cxr_test_csv, on=['Path'])

nih_dfs = []
for i in [1,2,3,4,5]:
    nih_dfs.append(pd.read_csv('model_files/nih_run_{}/nih_test_result.csv'.format(i)))
nih_df = pd.concat(nih_dfs)[['Path', 'Pneumothorax']]
nih_df.columns = ['Path', 'Pneumothorax_pred']
nih_df = nih_df.groupby(nih_df['Path']).mean()
nih_test_csv = pd.read_csv('nih_test_df.csv')
nih_df = nih_df.merge(nih_test_csv, on=['Path'])
nih_df['study_id'] = nih_df.apply(lambda row: str(row['Patient ID'])+'/'+str(row['Follow-up #']), axis=1)

cxp_dfs = []
for i in [1,2,3,4,5]:
    cxp_dfs.append(pd.read_csv('model_files/cxp_run_{}/cxp_test_result.csv'.format(i)))
cxp_df = pd.concat(cxp_dfs)[['Path', 'Pneumothorax']]
cxp_df.columns = ['Path', 'Pneumothorax_pred']
cxp_df = cxp_df.groupby(cxp_df['Path']).mean()
cxp_test_csv = pd.read_csv('cxp_test_df.csv')
cxp_df = cxp_df.merge(cxp_test_csv, on=['Path'])
cxp_df['study_id'] = cxp_df.apply(lambda row: row['Path'].split('chexpert_full/train/')[1].split('/view')[0], axis=1)


In [25]:
cxr = cxr_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
len(cxr)

23204

In [26]:
cxp = cxp_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
len(cxp)

18688

In [27]:
nih = nih_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
len(nih)

11196

In [5]:
cxr = cxr_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('CXR auc: {}'.format(bootstrap_auc(cxr['Pneumothorax'].values, cxr['Pneumothorax_pred'].values)))
gc.collect()

CXR auc: (0.8922053100633188, 0.009443761978620402)


20

In [6]:
cxp = cxp_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('CXP auc: {}'.format(bootstrap_auc(cxp['Pneumothorax'].values, cxp['Pneumothorax_pred'].values)))
gc.collect()

CXP auc: (0.9032517929665954, 0.008727204683319778)


0

In [7]:
nih = nih_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('NIH auc: {}'.format(bootstrap_auc(nih['Pneumothorax'].values, nih['Pneumothorax_pred'].values)))
gc.collect()

NIH auc: (0.8830817607306161, 0.014857936997854426)


0

In [8]:
# CXR ON NIH
cxr_dfs = []
for i in [1,2,3,4,5]:
    cxr_dfs.append(pd.read_csv('model_files/cxr_run_{}/nih_test_result.csv'.format(i)))
cxr_df = pd.concat(cxr_dfs)[['Path', 'Pneumothorax']]
cxr_df.columns = ['Path', 'Pneumothorax_pred']
cxr_df = cxr_df.groupby(cxr_df['Path']).mean()
cxr_df = cxr_df.merge(nih_test_csv, on=['Path'])
cxr_df['study_id'] = cxr_df.apply(lambda row: str(row['Patient ID'])+'/'+str(row['Follow-up #']), axis=1)

cxr = cxr_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('CXR on NIH auc: {}'.format(bootstrap_auc(cxr['Pneumothorax'].values, cxr['Pneumothorax_pred'].values)))
gc.collect()

# CXR ON CXP
cxr_dfs = []
for i in [1,2,3,4,5]:
    cxr_dfs.append(pd.read_csv('model_files/cxr_run_{}/cxp_test_result.csv'.format(i)))
cxr_df = pd.concat(cxr_dfs)[['Path', 'Pneumothorax']]
cxr_df.columns = ['Path', 'Pneumothorax_pred']
cxr_df = cxr_df.groupby(cxr_df['Path']).mean()
cxr_df = cxr_df.merge(cxp_test_csv, on=['Path'])
cxr_df['study_id'] = cxr_df.apply(lambda row: row['Path'].split('chexpert_full/train/')[1].split('/view')[0], axis=1)

cxr = cxr_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('CXR on CXP auc: {}'.format(bootstrap_auc(cxr['Pneumothorax'].values, cxr['Pneumothorax_pred'].values)))


CXR on NIH auc: (0.839208977202384, 0.02104083975347093)
CXR on CXP auc: (0.8265831697072394, 0.012005073235688724)


In [9]:
# NIH ON CXP
nih_dfs = []
for i in [1,2,3,4,5]:
    nih_dfs.append(pd.read_csv('model_files/nih_run_{}/cxp_test_result.csv'.format(i)))
nih_df = pd.concat(nih_dfs)[['Path', 'Pneumothorax']]
nih_df.columns = ['Path', 'Pneumothorax_pred']
nih_df = nih_df.groupby(nih_df['Path']).mean()
nih_df = nih_df.merge(cxp_test_csv, on=['Path'])
nih_df['study_id'] = nih_df.apply(lambda row: row['Path'].split('chexpert_full/train/')[1].split('/view')[0], axis=1)

nih = nih_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('NIH on CXP auc: {}'.format(bootstrap_auc(nih['Pneumothorax'].values, nih['Pneumothorax_pred'].values)))

gc.collect()
# NIH ON CXR
nih_dfs = []
for i in [1,2,3,4,5]:
    nih_dfs.append(pd.read_csv('model_files/nih_run_{}/cxr_test_result.csv'.format(i)))
nih_df = pd.concat(nih_dfs)[['Path', 'Pneumothorax']]
nih_df.columns = ['Path', 'Pneumothorax_pred']
nih_df = nih_df.groupby(nih_df['Path']).mean()
nih_df = nih_df.merge(cxr_test_csv, on=['Path'])

nih = nih_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('NIH on CXR auc:  {}'.format(bootstrap_auc(nih['Pneumothorax'].values, nih['Pneumothorax_pred'].values)))


NIH on CXP auc: (0.7788719890048941, 0.013854236904995265)
NIH on CXR auc:  (0.7589699379298408, 0.016889082124879884)


In [10]:
#CXP ON CXR
cxp_dfs = []
for i in [1,2,3,4,5]:
    cxp_dfs.append(pd.read_csv('model_files/cxp_run_{}/cxr_test_result.csv'.format(i)))
cxp_df = pd.concat(cxp_dfs)[['Path', 'Pneumothorax']]
cxp_df.columns = ['Path', 'Pneumothorax_pred']
cxp_df = cxp_df.groupby(cxp_df['Path']).mean()
cxp_df = cxp_df.merge(cxr_test_csv, on=['Path'])

cxp = cxp_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('CXP on CXR auc: {}'.format(bootstrap_auc(cxp['Pneumothorax'].values, cxp['Pneumothorax_pred'].values)))

gc.collect()
#CXP on NIH
cxp_dfs = []
for i in [1,2,3,4,5]:
    cxp_dfs.append(pd.read_csv('model_files/cxp_run_{}/nih_test_result.csv'.format(i)))
cxp_df = pd.concat(cxp_dfs)[['Path', 'Pneumothorax']]
cxp_df.columns = ['Path', 'Pneumothorax_pred']
cxp_df = cxp_df.groupby(cxp_df['Path']).mean()
cxp_df = cxp_df.merge(nih_test_csv, on=['Path'])
cxp_df['study_id'] = cxp_df.apply(lambda row: str(row['Patient ID'])+'/'+str(row['Follow-up #']), axis=1)

cxp = cxp_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
print('CXP on NIH auc: {}'.format(bootstrap_auc(cxp['Pneumothorax'].values, cxp['Pneumothorax_pred'].values)))



CXP on CXR auc: (0.8701812844562211, 0.012041924947213956)
CXP on NIH auc: (0.8520762312558305, 0.019684178779250383)


In [11]:
(0.009+0.012+0.013+0.012+0.009+0.016+0.020+0.021+0.015)/9

0.01411111111111111

In [12]:
((.903-.870)+(.903-.852)+(.892-.827)+(.892-.839)+(.883-.779)+(.883-.759))/6

0.0716666666666667